In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformer import TransformerConfig
from lm import LM

from data import get_tokenizers

In [7]:
config = TransformerConfig(d_model=128, n_layers=1, n_heads=4, max_len=47, dropout=0.)
model = LM(config, vocab_size=46)
model.load_state_dict(torch.load("transformer.pth"))

char_to_int, int_to_char = get_tokenizers()

In [305]:
def sample(self, prompt = "", g = torch.Generator()):
    idx = torch.tensor([char_to_int[c] for c in prompt], dtype=torch.int32).unsqueeze(0)
    idx = torch.cat([torch.tensor(char_to_int['<SOS>']).view(1, 1), idx], dim=1)
    next_id = -1

    while next_id != char_to_int['<EOS>']:
        act = self(idx, act=True) # (1, l, d_model)
        act[:, :, 56] = 3*0.8

        x = self.out_norm(act)
        logits = self.lm_head(x)

        probs = F.softmax(logits[:, -1, :], dim=-1)
        next_id = torch.multinomial(probs, num_samples=1, generator=g).item()
        idx = torch.cat([idx, torch.tensor(next_id).view(1, 1)], dim=1)

    print(act.max(dim=2).indices)
    #print(act[:, :, 21])
        
    return "".join([int_to_char[p.item()] for p in idx[0, 1:-1]])

In [314]:
sample(model, "")

tensor([[56, 56, 56, 56, 56, 56, 56, 56, 56]])


'écbechad'